# Trained ResNet-50 classification 

- new data generators

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import datetime
import numpy as np

In [3]:
# Project
import sys
sys.path.append(os.path.join(os.path.abspath(os.path.dirname('.')), '..', 'common'))

In [4]:
import matplotlib.pylab as plt
%matplotlib inline

def plt_st(n, m):
    plt.figure(figsize=(n, m))

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [5]:
from resnet_keras122 import get_resnet50

Using Theano backend.


Couldn't import dot_parser, loading of dot files will not be possible.


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GT 750M (CNMeM is enabled with initial size: 50.0% of memory, cuDNN 5103)


In [11]:
from training_utils import get_trainval_id_type_lists

print("\n {} - Get train/val lists ...".format(datetime.now()))
train_id_type_list, val_id_type_list = get_trainval_id_type_lists()


 2017-04-08 00:19:43.660835 - Get train/val lists ...
Train dataset contains : 
('-', [175, 546, 315], ' images of corresponding types')
Validation dataset contains : 
('-', [75, 234, 135], ' images of corresponding types')


In [17]:
print("\n {} - Get ResNet-50 model ...".format(datetime.now()))
resnet = get_resnet50(opt='adadelta')
resnet.summary()


 2017-04-08 00:53:22.905773 - Get ResNet-50 model ...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
zeropadding2d_3 (ZeroPadding2D)  (None, 3, 230, 230)   0           input_3[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 64, 112, 112)  9472        zeropadding2d_3[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 64, 112, 112)  256         conv1[0][0]                      
____________________________________

In [8]:
# from training_utils import exp_decay

# lr_1 = 0.5; a_1 = 0.915
# lr_2 = 1e-2; a_2 = 0.95

# epochs = np.arange(0, 50)
# lrs_1 = exp_decay(epochs, lr=lr_1, a=a_1)
# lrs_2 = exp_decay(epochs, lr=lr_2, a=a_2)
# plt_st(12, 5)
# plt.plot(epochs, lrs_1, label="lr=%f, a=%f" % (lr_1, a_1))
# plt.plot(epochs, lrs_2, label="lr=%f, a=%f" % (lr_2, a_2))
# plt.legend()

In [18]:
seed = 2017
np.random.seed(seed)
save_prefix='resnet_4_dense_adadelta_seed=%i' % seed

In [8]:
from glob import glob

weights_files = glob("weights/%s*.h5" % save_prefix)
best_val_loss = 1e5
best_weights_filename = ""
for f in weights_files:
    index = os.path.basename(f).index('-')
    loss = float(os.path.basename(f)[index+1:-4])
    if best_val_loss > loss:
        best_val_loss = loss
        best_weights_filename = f
print("Best val loss weights: ", best_weights_filename)


if len(best_weights_filename) > 0:
    # load weights to the model
    print("Load found weights: ", best_weights_filename)
    resnet.load_weights(best_weights_filename)

('Best val loss weights: ', '')


In [9]:
from xy_providers import DataCache  
cache = DataCache(2000)

In [ ]:
if True:
    from training_utils import classification_train as train
      
    nb_epochs = 50
    batch_size = 4
    
#     lrate_decay_f = lambda epoch: exp_decay(epoch, lr=lr_1, a=a_1) 

    print("\n {} - Start training ...".format(datetime.now()))
    h = train(resnet, 
              train_id_type_list, 
              val_id_type_list, 
              nb_epochs=nb_epochs,
              lrate_decay_f=None, #lrate_decay_f,
              batch_size=batch_size, 
              xy_provider_cache=cache,
              seed=seed,
              save_prefix=save_prefix)


 2017-04-08 00:53:45.810554 - Start training ...
('-- Training parameters: ', 4, 50, 2048, 1024)
-- Fit stats of train dataset
Load existing file: /Users/vfomin/Documents/ML/Kaggle/Intel_MobileODT/input/generated/resnet_4_dense_adadelta_seed=2017_stats.npz
No need to recompute statistics
-- Fit model
Epoch 1/50
2048/2048 [==============================] - 209s - loss: 1.2192 - acc: 0.3203 - val_loss: 1.0928 - val_acc: 0.3975
Epoch 2/50
2048/2048 [==============================] - 200s - loss: 1.1173 - acc: 0.3315 - val_loss: 1.0846 - val_acc: 0.4141
Epoch 3/50
2048/2048 [==============================] - 201s - loss: 1.1042 - acc: 0.3623 - val_loss: 1.0859 - val_acc: 0.3975
Epoch 4/50
2048/2048 [==============================] - 204s - loss: 1.0928 - acc: 0.3750 - val_loss: 1.0742 - val_acc: 0.4043
Epoch 5/50
  48/2048 [..............................] - ETA: 135s - loss: 1.0248 - acc: 0.4583

In [ ]:
batch_size = 4
print("\n {} - Start validation ...".format(datetime.datetime.now()))
validate(resnet, val_id_type_list, batch_size=batch_size)

In [ ]:
print("\n {} - Start predictions and write submission ...".format(datetime.datetime.now()))
predict(resnet, info=save_prefix, batch_size=batch_size)

## Notes

### Confinguration : 

- Train on the whole image
- Adadelta optimizer, lr=1.0, no learning rate decay
```
```

### Confinguration : 

- Train on the whole image
- Nadam optimizer. No learning rate decay
```
 2017-04-08 00:19:46.284410 - Start training ...
('-- Training parameters: ', 4, 50, 2048, 1024)
-- Fit stats of train dataset
Load existing file: /Users/vfomin/Documents/ML/Kaggle/Intel_MobileODT/input/generated/resnet_4_dense_nadam_seed=2017_stats.npz
No need to recompute statistics
-- Fit model
Epoch 1/50
2048/2048 [==============================] - 452s - loss: 1.2425 - acc: 0.3247 - val_loss: 1.0987 - val_acc: 0.3301
Epoch 2/50
2048/2048 [==============================] - 206s - loss: 1.1015 - acc: 0.3340 - val_loss: 1.0985 - val_acc: 0.3330
Epoch 3/50
2048/2048 [==============================] - 207s - loss: 1.0984 - acc: 0.3433 - val_loss: 1.0987 - val_acc: 0.3320
Epoch 4/50
2048/2048 [==============================] - 208s - loss: 1.0994 - acc: 0.3257 - val_loss: 1.0985 - val_acc: 0.3389
Epoch 5/50
2048/2048 [==============================] - 208s - loss: 1.1011 - acc: 0.3555 - val_loss: 1.0986 - val_acc: 0.3281
Epoch 6/50
2048/2048 [==============================] - 204s - loss: 1.1012 - acc: 0.3193 - val_loss: 1.0986 - val_acc: 0.3340
```

### Configuration :

- Train on the whole image
- Adadelta optimizer, lr=0.1, no learning rate decay
```
2017-04-07 20:23:18.792709 - Start training ...
('-- Training parameters: ', 4, 50, 2048, 1024)
-- Fit stats of train dataset
Load existing file: /Users/vfomin/Documents/ML/Kaggle/Intel_MobileODT/input/generated/resnet_4_dense_seed=2017_stats.npz
No need to recompute statistics
-- Fit model
Epoch 1/50
2048/2048 [] - 435s - loss: 1.2022 - acc: 0.3276 - val_loss: 1.1174 - val_acc: 0.3389
Epoch 2/50
2048/2048 [] - 198s - loss: 1.1641 - acc: 0.3345 - val_loss: 1.0938 - val_acc: 0.3770
Epoch 3/50
2048/2048 [] - 200s - loss: 1.1388 - acc: 0.3340 - val_loss: 1.0952 - val_acc: 0.3760
Epoch 4/50
2048/2048 [] - 207s - loss: 1.1172 - acc: 0.3481 - val_loss: 1.0940 - val_acc: 0.3779
Epoch 5/50
2048/2048 [] - 207s - loss: 1.1249 - acc: 0.3423 - val_loss: 1.0926 - val_acc: 0.3740
Epoch 6/50
2048/2048 [] - 202s - loss: 1.1096 - acc: 0.3530 - val_loss: 1.0889 - val_acc: 0.3828
Epoch 7/50
2048/2048 [] - 204s - loss: 1.1150 - acc: 0.3447 - val_loss: 1.0926 - val_acc: 0.3760
Epoch 8/50
2048/2048 [] - 219s - loss: 1.1030 - acc: 0.3530 - val_loss: 1.0847 - val_acc: 0.3896
Epoch 9/50
2048/2048 [] - 213s - loss: 1.1093 - acc: 0.3511 - val_loss: 1.0841 - val_acc: 0.4141
Epoch 10/50
2048/2048 [] - 210s - loss: 1.0976 - acc: 0.3628 - val_loss: 1.0828 - val_acc: 0.4199
Epoch 11/50
2048/2048 [] - 206s - loss: 1.1029 - acc: 0.3672 - val_loss: 1.0845 - val_acc: 0.4043
Epoch 12/50
2048/2048 [] - 199s - loss: 1.0941 - acc: 0.3789 - val_loss: 1.0775 - val_acc: 0.4463
Epoch 13/50
2048/2048 [] - 203s - loss: 1.0953 - acc: 0.3779 - val_loss: 1.0710 - val_acc: 0.4658
Epoch 14/50
2048/2048 [] - 205s - loss: 1.0884 - acc: 0.3823 - val_loss: 1.0657 - val_acc: 0.4521
Epoch 15/50
2048/2048 [] - 208s - loss: 1.0863 - acc: 0.3931 - val_loss: 1.0707 - val_acc: 0.4375
Epoch 16/50
2048/2048 [] - 204s - loss: 1.0878 - acc: 0.3799 - val_loss: 1.0696 - val_acc: 0.4551
Epoch 17/50
2048/2048 [] - 208s - loss: 1.0815 - acc: 0.3936 - val_loss: 1.0651 - val_acc: 0.4561
Epoch 18/50
2048/2048 [] - 207s - loss: 1.0856 - acc: 0.4043 - val_loss: 1.0627 - val_acc: 0.4561
Epoch 19/50
2048/2048 [] - 203s - loss: 1.0784 - acc: 0.3975 - val_loss: 1.0539 - val_acc: 0.4492
Epoch 20/50
2048/2048 [] - 209s - loss: 1.0740 - acc: 0.4072 - val_loss: 1.0409 - val_acc: 0.4873
Epoch 21/50
2048/2048 [] - 213s - loss: 1.0783 - acc: 0.4019 - val_loss: 1.0474 - val_acc: 0.4795
Epoch 22/50
2048/2048 [] - 210s - loss: 1.0697 - acc: 0.4058 - val_loss: 1.0514 - val_acc: 0.4521
Epoch 23/50
2048/2048 [] - 216s - loss: 1.0668 - acc: 0.4185 - val_loss: 1.0386 - val_acc: 0.4688
Epoch 24/50
2048/2048 [] - 203s - loss: 1.0771 - acc: 0.3970 - val_loss: 1.0472 - val_acc: 0.4668
Epoch 25/50
2048/2048 [] - 216s - loss: 1.0609 - acc: 0.4214 - val_loss: 1.0414 - val_acc: 0.4648
Epoch 26/50
2048/2048 [] - 208s - loss: 1.0642 - acc: 0.4175 - val_loss: 1.0404 - val_acc: 0.4609
Epoch 27/50
2048/2048 [] - 215s - loss: 1.0573 - acc: 0.4268 - val_loss: 1.0344 - val_acc: 0.4678
Epoch 28/50
2048/2048 [] - 206s - loss: 1.0613 - acc: 0.4155 - val_loss: 1.0282 - val_acc: 0.4717
Epoch 29/50
2048/2048 [] - 210s - loss: 1.0414 - acc: 0.4497 - val_loss: 1.0314 - val_acc: 0.4600
Epoch 30/50
2048/2048 [] - 217s - loss: 1.0554 - acc: 0.4321 - val_loss: 1.0219 - val_acc: 0.4795
Epoch 31/50
2048/2048 [] - 202s - loss: 1.0468 - acc: 0.4336 - val_loss: 1.0089 - val_acc: 0.4902
Epoch 32/50
2048/2048 [] - 210s - loss: 1.0416 - acc: 0.4390 - val_loss: 1.0135 - val_acc: 0.4932
Epoch 33/50
2048/2048 [] - 202s - loss: 1.0396 - acc: 0.4507 - val_loss: 1.0096 - val_acc: 0.4951
Epoch 34/50
2048/2048 [] - 203s - loss: 1.0376 - acc: 0.4556 - val_loss: 1.0107 - val_acc: 0.4727
Epoch 35/50
2048/2048 [] - 211s - loss: 1.0300 - acc: 0.4551 - val_loss: 1.0098 - val_acc: 0.4902
Epoch 36/50
2048/2048 [] - 203s - loss: 1.0334 - acc: 0.4546 - val_loss: 1.0075 - val_acc: 0.4844
```


### Configuration :

- Train on the whole image
- No featurewise mean/std
- Adadelta optimizer, lr=0.1, no learning rate decay
```
 2017-04-07 18:21:49.271054 - Start training ...
('-- Training parameters: ', 4, 50, 2048, 1024)
-- Fit model
Epoch 1/50
2048/2048 [] - 204s - loss: 1.2735 - acc: 0.3291 - val_loss: 1.1178 - val_acc: 0.2988
Epoch 2/50
2048/2048 [] - 216s - loss: 1.1677 - acc: 0.3379 - val_loss: 1.1024 - val_acc: 0.3340
Epoch 3/50
2048/2048 [] - 208s - loss: 1.1499 - acc: 0.3442 - val_loss: 1.0949 - val_acc: 0.3682
Epoch 4/50
2048/2048 [] - 208s - loss: 1.1364 - acc: 0.3354 - val_loss: 1.0960 - val_acc: 0.3604
Epoch 5/50
2048/2048 [] - 211s - loss: 1.1183 - acc: 0.3618 - val_loss: 1.0953 - val_acc: 0.3564
Epoch 6/50
2048/2048 [] - 209s - loss: 1.1322 - acc: 0.3184 - val_loss: 1.0919 - val_acc: 0.3984
Epoch 7/50
2048/2048 [] - 208s - loss: 1.1119 - acc: 0.3574 - val_loss: 1.0972 - val_acc: 0.3662
Epoch 8/50
2048/2048 [] - 209s - loss: 1.1134 - acc: 0.3486 - val_loss: 1.0958 - val_acc: 0.3613
Epoch 9/50
2048/2048 [] - 217s - loss: 1.1091 - acc: 0.3384 - val_loss: 1.0908 - val_acc: 0.3867
Epoch 10/50
2048/2048 [] - 210s - loss: 1.1036 - acc: 0.3569 - val_loss: 1.0898 - val_acc: 0.3975
Epoch 11/50
2048/2048 [] - 208s - loss: 1.1012 - acc: 0.3599 - val_loss: 1.0857 - val_acc: 0.4180
Epoch 12/50
2048/2048 [] - 204s - loss: 1.1062 - acc: 0.3535 - val_loss: 1.0905 - val_acc: 0.3740
Epoch 13/50
2048/2048 [] - 211s - loss: 1.0994 - acc: 0.3599 - val_loss: 1.0884 - val_acc: 0.3896
Epoch 14/50
2048/2048 [] - 203s - loss: 1.1001 - acc: 0.3667 - val_loss: 1.0915 - val_acc: 0.3848
Epoch 15/50
2048/2048 [] - 200s - loss: 1.0957 - acc: 0.3672 - val_loss: 1.0890 - val_acc: 0.3760
Epoch 16/50
2048/2048 [] - 206s - loss: 1.0978 - acc: 0.3750 - val_loss: 1.0908 - val_acc: 0.3906
Epoch 17/50
2048/2048 [] - 200s - loss: 1.0953 - acc: 0.3608 - val_loss: 1.0815 - val_acc: 0.4111
Epoch 18/50
2048/2048 [] - 200s - loss: 1.0947 - acc: 0.3564 - val_loss: 1.0861 - val_acc: 0.3877
Epoch 19/50
2048/2048 [] - 200s - loss: 1.0942 - acc: 0.3696 - val_loss: 1.0855 - val_acc: 0.4053
Epoch 20/50
2048/2048 [] - 200s - loss: 1.0902 - acc: 0.3594 - val_loss: 1.0872 - val_acc: 0.3965
Epoch 21/50
2048/2048 [] - 200s - loss: 1.0942 - acc: 0.3857 - val_loss: 1.0813 - val_acc: 0.4268
Epoch 22/50
2048/2048 [] - 200s - loss: 1.0955 - acc: 0.3472 - val_loss: 1.0768 - val_acc: 0.4170
Epoch 23/50
2048/2048 [] - 200s - loss: 1.0812 - acc: 0.3945 - val_loss: 1.0744 - val_acc: 0.4170
Epoch 24/50
2048/2048 [] - 200s - loss: 1.0891 - acc: 0.3809 - val_loss: 1.0732 - val_acc: 0.4551
Epoch 25/50
2048/2048 [] - 210s - loss: 1.0801 - acc: 0.3989 - val_loss: 1.0662 - val_acc: 0.4248
Epoch 26/50
2048/2048 [] - 211s - loss: 1.0814 - acc: 0.3877 - val_loss: 1.0669 - val_acc: 0.4424
Epoch 27/50
2048/2048 [] - 204s - loss: 1.0798 - acc: 0.3975 - val_loss: 1.0602 - val_acc: 0.4512
Epoch 28/50
2048/2048 [] - 201s - loss: 1.0673 - acc: 0.4033 - val_loss: 1.0633 - val_acc: 0.4326
Epoch 29/50
2048/2048 [] - 204s - loss: 1.0717 - acc: 0.4233 - val_loss: 1.0545 - val_acc: 0.4482
Epoch 30/50
2048/2048 [] - 205s - loss: 1.0751 - acc: 0.4116 - val_loss: 1.0568 - val_acc: 0.4639
Epoch 31/50
2048/2048 [] - 204s - loss: 1.0639 - acc: 0.4165 - val_loss: 1.0535 - val_acc: 0.4648
Epoch 32/50
2048/2048 [] - 197s - loss: 1.0631 - acc: 0.4404 - val_loss: 1.0495 - val_acc: 0.4443
```

